# Код для парсинга профилей и постов LinkedIn

In [ ]:
import time
import configparser
import random
import re
import os.path

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

**Загружаем конфиг**

In [ ]:
# путь к файлу расширения для Chrome "Доступ к LinkedIn"
EXTENSION_PATH = '1.5_0.crx'

# конфиг
CFG_FILE = 'parser.ini'

DATA_PATH = '../datasets/'

"""
файл конфигурации необходимо предварительно создать,
формат файла parser.ini:
[LINKEDIN]
USER_LOGIN = эл_почта_без_кавычек
USER_PASSWORD = пароль
""";

# загружаем данные из конфига
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    print(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')

**Общие процедуры и функции**

In [ ]:
# прокрутка страницы, для подгрузки динамического контента
def get_scrolled_page(driver, num_scrolls=15, pause_time=0.5):
    """
    Функция прокручивает страницу, загруженную в экземпляр driver,
    num_scrolls раз, с pause_time паузами между прокрутками.
    Возвращает код страницы.
    """
    # текущая высота body
    last_height = driver.execute_script('return document.body.scrollHeight')
    for i in range(num_scrolls):
        
        # нажимаем кнопку PageDown 5 раз
        for _ in range(5):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            # делаем паузу для загрузки динамического контента
            time.sleep(random.uniform(pause_time, 3))
        
        # вычисляем новую высоту body
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break 
        last_height = new_height

    return driver

In [ ]:
# собираем информацию о пользователе
def get_user_info(driver, user_id):
    """
    Функция парсит со страницы профиля информацию о пользователе.
    На вход получает, драйвер и идентификатор пользователя.
    На выходе возвращает список с данным профиля
    """
    # прокручиваем страницу до конца что бы подгрузился динамический контент
    driver = get_scrolled_page(driver, num_scrolls=3, pause_time=0.5)

    # извлекаем код страницы
    src = driver.page_source

    # предеаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')    
    
    # извлекаем HTML содержаший имя и заголовок
    intro = soup.find('div', {'class': 'mt2 relative'})

    # получаем имя
    user_name = ''
    try:
        name_loc = intro.find("h1")
        user_name = name_loc.get_text().strip()
    except: ...

    # заголовок, обычно тут пишут где работает или специальность или навыки
    user_head = ''
    try:
        head_at_loc = intro.find("div", {'class': 'text-body-medium'})
        user_head = head_at_loc.get_text().strip()
    except: ...

    # получаем теги
    user_tags = ''
    try:
        # темы публикаций
        tags_at_loc = intro.find("div", {'class': 'text-body-small t-black--light break-words mt2'})
        # уточняем 
        tags_at_loc = tags_at_loc.find('span', {'aria-hidden': 'true'})
        # убираем лишние символы
        user_tags = tags_at_loc.get_text().split(':')[1].strip()
        user_tags = user_tags.replace('#','').replace(' и',',')
    except: ...

    # получаем локацию пользователя
    user_location = ''
    try:
        location_at_loc = intro.find("div", {'class': 'pv-text-details__left-panel mt2'})
        # уточняем
        location_at_loc = location_at_loc.find('span', {'class': 'text-body-small'})
        user_location = location_at_loc.get_text().strip()
    except: ...

    # место работы
    user_work = ''
    try:
        work_at_loc = intro.find("div", {'class': 'inline-show-more-text'})
        user_work = work_at_loc.get_text().strip()
    except: ...

    # количество отслеживающих и контактов
    user_viewwers, user_contacts = '0', '0'
    try:
        stat_at_loc = soup.find("ul", {'class': 'pv-top-card--list pv-top-card--list-bullet'})
        user_viewwers = stat_at_loc.find_all("span")[0].get_text().strip()
        user_contacts = stat_at_loc.find_all("span")[2].get_text().strip()
    except: ...

    # общие сведения
    user_common_info = ''
    try:
        common_at_loc = soup.find("div", {'class': 'display-flex ph5 pv3'})
        user_common_info = common_at_loc.find_all('span')[0].get_text().strip()
    except: ...

    # должность
    user_position = ''
    try:
        position_at_loc = soup.find("ul", {'class': 'pvs-list'})
        user_position = position_at_loc.find_all('span')[0].get_text().strip()
    except: ...
        
    return [
        user_name, user_head, user_work, user_position, user_tags,
        user_location, user_viewwers, user_contacts, user_common_info
    ]

In [ ]:
# парсим данные публикации
def get_post_info(post):
    """
    Функция на вход получает блок кода с публикацией.
    Возвращает список параметров публикации: текст и реакции.
    """
    # текст поста
    post_text = 'no text'
    try:
        post_text = post.find('span', {'class': 'break-words'}).get_text().strip()
    except: ...

    # блок реакций на пост
    likes, comments, reposts = '0', '0', '0'
    try:
        reaсtions = post.find('ul', {'class': 'social-details-social-counts'})
        try:
            likes = reaсtions.find(
                'span', {'class': 'social-details-social-counts__reactions-count'}
            ).get_text().strip().replace('\xa0', ' ')
            
        except: ...
        try:
            comments = reaсtions.find(
                'li', {'class': 'social-details-social-counts__comments'}
            ).get_text().strip().replace('\xa0', ' ')
            comments = re.match('^[\d]+', comments)[0]
        except: ...
        try:
            reposts = reaсtions.find(
                'li', {'class': 'social-details-social-counts__item social-details-social-counts__item--with-social-proof'}
            ).get_text().strip().replace('\xa0', ' ')
            reposts = re.match('^[\d]+', reposts)[0]
        except: ...
    except: ...
        
    return [post_text, likes, comments, reposts]

**Создаем и запускаем браузер**

In [ ]:
# подключаем расширение к драйверу
options = webdriver.ChromeOptions()
options.add_extension(EXTENSION_PATH)

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
options.page_load_strategy = 'eager'

# запускаем Chrome с расширением
driver = webdriver.Chrome(options=options)

**Вход в LinkedIn**

In [ ]:
# открываем страницу входа linkedIn, необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(3)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")
# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")
# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
driver.find_element(By.XPATH, "//button[@type='submit']").click()

**Собираем все id в один датафрейм**

In [ ]:
# имя файла для сохранения профилей юзеров
CSV_PROFILES_FILE_NAME = os.path.join(DATA_PATH, 'profiles.csv')

# названия столбцов для хранения данных о пользователях
profile_columns = [
    'user_name', # имя
    'user_head', # заголовок
    'user_work', # последннее/текущее место работы
    'user_position', # должность
    'user_tags', # теги, интересы
    'user_location', # адрес
    'user_viewers', # число подписчиков
    'user_contacts', # число контактов
    'user_common_info' # общая информация
]

In [ ]:
# если файл с профилями уже существует
if os.path.exists(CSV_PROFILES_FILE_NAME):
    # загружаем датафрейм из файла
    df = pd.read_csv(CSV_PROFILES_FILE_NAME, index_col=0)
else:
    # список файлов c id пользователей
    list_csv_files = [
        'profiles_id_1.csv',
        'profiles_id_2.csv',
        'profiles_id_3.csv',
        'profiles_id_4.csv',
        'profiles_id_5.csv',
    ]
    # пустой DF
    df = pd.DataFrame(columns=['id'])

    # соберем все файлы в один DF
    for csv_file in list_csv_files:
        csv_file_name = os.path.join(DATA_PATH, csv_file)
        df = pd.concat(
            [df, pd.read_csv(csv_file_name, index_col=0)]
        ).reset_index(drop=True)

    # удаляем дубли
    df = df.drop_duplicates()

    df = df.reindex(columns = df.columns.tolist() + profile_columns)

print('Всего профилей:', len(df))

In [ ]:
df.info()

**Парсим профили и посты**

In [ ]:
# имя файла для сохранения публикаций
CSV_POSTS_FILE_NAME = os.path.join(DATA_PATH, 'posts.csv')

# названия столбцов для хранения публикаций
posts_columns = [
    'user_id', # id профиля
    'text', # текст публикации
    'likes', # количество реакций
    'comments', # количество комментариев
    'reposts', # количество комментариев
]

In [ ]:
# если файл с профилями уже существует
if os.path.exists(CSV_POSTS_FILE_NAME):
    # загружаем датафрейм из файла
    df_posts = pd.read_csv(CSV_POSTS_FILE_NAME, index_col=0)
else:
    # пустой датафрейм для текстов публикаций
    df_posts = pd.DataFrame(columns=posts_columns)

In [ ]:
df_posts.info()

In [ ]:
# с какого профиля стартуем
# если ранее парсинг был прерван, продолжаем с того же места
start_idx = df.user_name.nunique()
start_idx

In [ ]:
# парсим данные из профилей
for profile_id in df.id[start_idx:]:
    
    # для контроля выводим на экран текущий ID профиля
    print(profile_id)
    
    # получаем url профиля пользователя
    profile_url = f'https://www.linkedin.com/in/{profile_id}/'

    # открываем ссылку profile_url
    driver.get(profile_url)

    # парсим информацию профиля
    user_info = get_user_info(driver, profile_id)
    
    # сохраняем данные в датафрейм
    df.loc[df.id == profile_id, profile_columns] = user_info
    
    # сохраняем в CSV
    df.to_csv(CSV_PROFILES_FILE_NAME)
    
    # пауза
    time.sleep(random.uniform(10, 20))
    
    # URL на все публикации пользователя
    posts_url = f'https://www.linkedin.com/in/{profile_id}/recent-activity/all/'

    driver.get(posts_url)

    # получаем код проскроленой страницы
    src = get_scrolled_page(driver, num_scrolls=25, pause_time=0.5).page_source

    # передаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')

    # получаем список постов
    posts_block = soup.find_all(
        'li', {'class': 'profile-creator-shared-feed-update__container'}
    )

    print(f'posts: {len(posts_block)}')

    count_posts = 1
    
    # парсим посты
    for post in posts_block:
        
        # номер поста для контроля
        print(count_posts, end=' ')
        count_posts += 1
        
        # получаем данные публикации
        post_info = get_post_info(post)
        
        if not post_info[0] == 'no text':
            # добавляем данные в датафрейм
            df_posts.loc[len(df_posts.index)] = [profile_id] + post_info
        
        # сохраняем в CSV
        df_posts.to_csv(CSV_POSTS_FILE_NAME)
    
    print()

In [ ]:
# закрываем браузер
driver.quit()

**Результат**

In [ ]:
# профили
df.info()

In [ ]:
# публикации
df_posts.info()